# Week 2 — Forward & Inverse Kinematics

**Learning objectives:**
- Derive forward kinematics for a planar 2-link manipulator.
- Compute analytic inverse kinematics and understand multiple solutions.

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Forward kinematics for 2-link planar arm
def fk_2link(theta1, theta2, L1=1.0, L2=0.8):
    x = L1*np.cos(theta1) + L2*np.cos(theta1+theta2)
    y = L1*np.sin(theta1) + L2*np.sin(theta1+theta2)
    return x, y

# Analytic inverse kinematics (elbow-up)
def ik_2link(x, y, L1=1.0, L2=0.8):
    D = (x**2 + y**2 - L1**2 - L2**2)/(2*L1*L2)
    if abs(D) > 1.0:
        raise ValueError('Target unreachable')
    th2 = np.arctan2(np.sqrt(max(0,1-D**2)), D)
    th1 = np.arctan2(y, x) - np.arctan2(L2*np.sin(th2), L1+L2*np.cos(th2))
    return th1, th2

# Test
x,y = fk_2link(0.5, 0.3)
print('FK ->', x, y)
th1,th2 = ik_2link(x,y)
print('Recovered IK ->', th1, th2)


### Exercises
- Plot workspace and reachable region.
- Implement numerical IK (Newton) using Jacobian inverse.
- Compare analytic and numeric IK results.